# Receipt OCR  

### ressources

http://asprise.com/ocr/api/docs/html/receipt-ocr.html <br/>
https://asprise.com/receipt-ocr-data-capture-api/extract-text-reader-scanner-index.html

In [1]:
import requests
import pandas as pd
import json
import seaborn as sns

print("=== Python Receipt OCR using API")

=== Python Receipt OCR using API


In [6]:
receiptOcrEndpoint = 'https://ocr.asprise.com/api/v1/receipt' # Receipt OCR API endpoint
imageFile = "img/1.jpg" # // Modify this to use your own file if necessary
r = requests.post(receiptOcrEndpoint, data = { \
  'client_id': 'TEST',        # Use 'TEST' for testing purpose \
  'recognizer': 'auto',       # can be 'US', 'CA', 'JP', 'SG' or 'auto' \
  'ref_no': 'ocr_python_123', # optional caller provided ref code \
  }, \
  files = {"file": open(imageFile, "rb")})

# Writing to sample.json
with open("sample.json", "w") as outfile:
    json.dump(r.json(), outfile)
    #https://www.geeksforgeeks.org/reading-and-writing-json-to-a-file-in-python/

print(r.text) # result in JSON


{
  "ref_no" : "ocr_python_123",
  "success" : false,
  "message" : "Daily quota exceeded. Try again in a few hours or contact us to increase the quota: ocr@asprise.com",
  "get_help" : "Please contact support@asprise.com for assistance."
}


In [7]:
with open("sample.json", "r") as openfile:
    #reading from json file
    json_object = json.load(openfile)

print(json_object)

{'ref_no': 'ocr_python_123', 'success': False, 'message': 'Daily quota exceeded. Try again in a few hours or contact us to increase the quota: ocr@asprise.com', 'get_help': 'Please contact support@asprise.com for assistance.'}


### Find all the items in the receipt

In [8]:
if json_object["receipts"] is not None: 
    receipt_object = json_object["receipts"]
    # Access the "items" list within the "receipts" object
    items = json_object["receipts"][0]["items"]
    print(items)
else:
    print("Error is"+json_object["message"])

KeyError: 'receipts'

In [ ]:
#iterate over each item and add them if flaf != none
product = []

for item in items:
    flag = item["flags"]
    if flag != "":
        product.append(item)
    #elif :
        #find total price
    else:
        #nothing to do
        print()
print(product)





[{'amount': 1.36, 'category': None, 'description': 'Bananen kg', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 2.99, 'category': None, 'description': 'Zwiebeln gelb', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 1.39, 'category': None, 'description': 'Mango Stk', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 1.99, 'category': None, 'description': 'Pfirsiche', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 1.99, 'category': None, 'description': 'Asiatisches Gemüse', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 1.59, 'category': None, 'description': 'Bioland Type 630', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 0.95, 'category': None, 'description': 'H-Milch 1,5% OGT', 'flags': ' A', 'qty': None, 'remarks': None, 'tags':

### Translation

Ressources : https://www.thepythoncode.com/article/translate-text-in-python

In [ ]:
from googletrans import Translator

#init the Google API translator
translator = Translator()

translated_text = translator.translate('안녕하세요.')
print(translated_text.text)

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
for item in (items, product): 
    description = item["description"]
    translation = translator.translate(description, dest="fr")
    item["description"] = translation.text

for item in items:
    print("Description : ", items["description"])

NameError: name 'items' is not defined

### Pandas dataframe

In [ ]:
items_df = pd.DataFrame(items)
product_df = pd.DataFrame(product)